In [ ]:
#coding.utf-8
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import re #정규식 표현을 위한 모듈
from crawling import *

In [ ]:
epi = []
stop = False
pageURL = "https://comic.naver.com/webtoon/list?titleId=736277&weekday=sun&page="
for i in range(1,10):
    for j in range(10):
        source_url = '{0}{1}'.format(pageURL,i)
        soup = html_parser(source_url)
        year = soup.select("td.num")[j].text.split(".")[0]
        if  (year == None) or (year == '2020'):
            stop = True
            break
        epi.append(soup.select("td.title > a")[j])
    if stop == True:
        break

In [ ]:
commentArray = []
url = 'https://comic.naver.com'
progress = ProgressBar(len(epi))
for i in epi:
    progress.next()
    soup = html_parser(url+i.attrs["href"])
    commentURL = soup.select_one("#commentIframe").attrs["src"]
    soup = html_parser(url+commentURL)
    comment = soup.select(".u_cbox_contents")
    commentArray.append(get_text(comment))
    print(get_text(comment))

In [ ]:
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-힣 ]+')
    result = hangul.sub('',text)
    return result

# Text = "".join(commentArray).replace('\n'," ")
# print(text_cleaning(Text))

In [ ]:
import konlpy

from konlpy.tag import Okt
from collections import Counter

with open("test.txt", encoding="utf8") as test:
    Text = "".join(test.readlines())
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(text_cleaning(Text))
count = Counter(nouns)

In [ ]:
len(count)

In [ ]:
len([i for i in count if len(i)>1])

In [ ]:
remove_char_counter = Counter({x:count[x] for x in count if len(x) >1})
korean_stopwords_path = 'korean_stopwords.txt'

# 텍스트 파일을 오픈 
with open(korean_stopwords_path,encoding = 'utf8') as f: 
    stopwords = f.readlines() 
stopwords = [x.strip() for x in stopwords]

remove_char_counter=Counter({x:remove_char_counter[x] for x in count if x not in stopwords})
remove_char_counter=Counter({x:remove_char_counter[x] for x in remove_char_counter if len(x)>1})
print(remove_char_counter)

In [ ]:
import random
import pytagcloud
import webbrowser

ranked_tags=remove_char_counter.most_common(50)
taglist = pytagcloud.make_tags(ranked_tags,maxsize=80)
pytagcloud.create_tag_image(taglist,'wordcloud.jpg',size=(900,600),fontname="NanumGothic",rectangular=False)
from IPython.display import Image
Image(filename='wordcloud.jpg')